In [1]:
import pandas as pd
import numpy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
from platform import python_version
import  matplotlib,torch,torchtext
!pip install portalocker>=2.0.0
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from collections import Counter
from collections import OrderedDict
from torchtext.vocab import vocab
import torch.nn as nn
from tqdm import tqdm
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
dataframe = pd.read_csv('/content/samsum-train.csv',engine='python',encoding='utf-8', error_bad_lines=True)

<ipython-input-2-b5e891d57e74>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframe = pd.read_csv('/content/samsum-train.csv',engine='python',encoding='utf-8', error_bad_lines=True)


In [3]:
def tokenizer(text):
    # step 1. remove HTML tags. they are not helpful in understanding the sentiments of a review
    # step 2: use lowercase for all text to keep symmetry
    # step 3: extract emoticons. keep them as they are important sentiment signals
    # step 4: remove punctuation marks
    # step 5: put back emoticons
    # step 6: generate word tokens
    text = re.sub("<[^>]*>", "", text)
    text = text.lower()
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = re.sub("[\W]+", " ", text)
    text = text + " ".join(emoticons).replace("-", "")
    tokenized = text.split()
    return tokenized

In [4]:
example_tokens=tokenizer(dataframe.dialogue[0])
example_tokens

['amanda',
 'i',
 'baked',
 'cookies',
 'do',
 'you',
 'want',
 'some',
 'jerry',
 'sure',
 'amanda',
 'i',
 'll',
 'bring',
 'you',
 'tomorrow',
 ':)']

In [5]:
token_counts = Counter()
token_counts.update(example_tokens)
token_counts

Counter({'amanda': 2,
         'i': 2,
         'baked': 1,
         'cookies': 1,
         'do': 1,
         'you': 2,
         'want': 1,
         'some': 1,
         'jerry': 1,
         'sure': 1,
         'll': 1,
         'bring': 1,
         'tomorrow': 1,
         ':)': 1})

In [6]:
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
sorted_by_freq_tuples

[('amanda', 2),
 ('i', 2),
 ('you', 2),
 ('baked', 1),
 ('cookies', 1),
 ('do', 1),
 ('want', 1),
 ('some', 1),
 ('jerry', 1),
 ('sure', 1),
 ('ll', 1),
 ('bring', 1),
 ('tomorrow', 1),
 (':)', 1)]

In [7]:
ordered_dict = OrderedDict(sorted_by_freq_tuples)
ordered_dict

OrderedDict([('amanda', 2),
             ('i', 2),
             ('you', 2),
             ('baked', 1),
             ('cookies', 1),
             ('do', 1),
             ('want', 1),
             ('some', 1),
             ('jerry', 1),
             ('sure', 1),
             ('ll', 1),
             ('bring', 1),
             ('tomorrow', 1),
             (':)', 1)])

In [8]:
len(ordered_dict)

14

In [9]:
vb = vocab(ordered_dict)
vb.get_stoi()

{'amanda': 0,
 'i': 1,
 ':)': 13,
 'you': 2,
 'baked': 3,
 'cookies': 4,
 'tomorrow': 12,
 'do': 5,
 'some': 7,
 'want': 6,
 'jerry': 8,
 'sure': 9,
 'll': 10,
 'bring': 11}

In [10]:
dataframe['dialogue'].isnull().sum()

1

In [11]:
dataframe = dataframe.dropna(subset=['dialogue'])

### pre-processing for dialogue

In [12]:
# step 1: convert reviews into tokens
# step 2: find frequency of tokens

token_counts_dialogue = Counter()

for i in range(0,len(dataframe)):
    tokens_dia = tokenizer(dataframe['dialogue'].iloc[i])
    token_counts_dialogue.update(tokens_dia)


print('dialogue vocab size:', len(token_counts_dialogue))

dialogue vocab size: 29629


In [13]:
# step 3: sort the token based on their frequency
# step 4: put the sorted tokens in OrderedDict
# step 5: convert token to integers using vocab object

sorted_by_freq_tuples = sorted(token_counts_dialogue.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vb = vocab(ordered_dict)

vb.insert_token("<pad>", 0)    # Padding token
vb.insert_token("<unk>", 1)    # Unknown token
vb.insert_token("<start>", 2)  # Start of sequence token
vb.insert_token("<end>", 3)    # End of sequence token

vb.set_default_index(1)  # Default index for unknown words

# print some token indexes from vocab
for token in ["<start>", "this", "is", "<end>"]:
    print(token, " --> ", vb[token])

<start>  -->  2
this  -->  40
is  -->  16
<end>  -->  3


### Pre-processing for Summary

In [14]:
# step 1: convert reviews into tokens
# step 2: find frequency of tokens

token_counts_summary = Counter()

for i in range(0,len(dataframe)):
    tokens_summary = tokenizer(dataframe['summary'].iloc[i])
    token_counts_summary.update(tokens_summary)


print('dialogue Summary size:', len(token_counts_summary))

dialogue Summary size: 16057


In [15]:
# step 3: sort the token based on their frequency
# step 4: put the sorted tokens in OrderedDict
# step 5: convert token to integers using vocab object

sorted_by_freq_tuples_summary = sorted(token_counts_summary.items(), key=lambda x: x[1], reverse=True)
ordered_dict_summary = OrderedDict(sorted_by_freq_tuples_summary)

vb_summary = vocab(ordered_dict_summary)

vb_summary.insert_token("<pad>", 0)    # Padding token
vb_summary.insert_token("<unk>", 1)    # Unknown token
vb_summary.insert_token("<start>", 2)  # Start of sequence token
vb_summary.insert_token("<end>", 3)    # End of sequence token

vb_summary.set_default_index(1)  # Default index for unknown words

# print some token indexes from vocab
for token in ["<start>", "this", "is", "<end>"]:
    print(token, " --> ", vb[token])

<start>  -->  2
this  -->  40
is  -->  16
<end>  -->  3


In [16]:
# inline lambda functions for text and label precessing
"""dialogue_pipeline = lambda x: [vb[token] for token in tokenizer(x)]
summary_pipeline = lambda x: [vb_summary[token] for token in tokenizer(x)]"""

dialogue_pipeline = lambda x: [vb["<start>"]] + [vb[token] for token in tokenizer(x)] + [vb["<end>"]]
summary_pipeline = lambda x: [vb_summary["<start>"]] + [vb_summary[token] for token in tokenizer(x)] + [vb_summary["<end>"]]


In [17]:
dataframe.columns

Index(['id', 'dialogue', 'summary'], dtype='object')

In [18]:
dataframe.dialogue[0]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"

In [19]:
dialogue_pipeline(dataframe.dialogue[0])

[2, 421, 4, 3947, 1830, 30, 5, 86, 61, 611, 62, 421, 4, 33, 257, 5, 121, 58, 3]

In [20]:
dataframe.summary[0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [21]:
summary_pipeline(dataframe.summary[0])

[2, 366, 2238, 967, 6, 9, 93, 546, 44, 41, 3]

In [22]:
train_data_vect_dialogue = []

for i in range(0,len(dataframe['dialogue'])):
  out = dialogue_pipeline(dataframe.dialogue.iloc[i])
  train_data_vect_dialogue.append(out)

In [23]:
train_data_vect_summary = []

for i in range(0,len(dataframe['summary'])):
  out = summary_pipeline(dataframe.summary.iloc[i])
  train_data_vect_summary.append(out)

In [24]:
train_data_vect_dialogue[0]

[2, 421, 4, 3947, 1830, 30, 5, 86, 61, 611, 62, 421, 4, 33, 257, 5, 121, 58, 3]

In [25]:
train_data_vect_summary[0]

[2, 366, 2238, 967, 6, 9, 93, 546, 44, 41, 3]

In [26]:
test_dataframe = pd.read_csv('/content/samsum-test.csv',engine='python',encoding='utf-8', error_bad_lines=True)

<ipython-input-26-2e08d2132617>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test_dataframe = pd.read_csv('/content/samsum-test.csv',engine='python',encoding='utf-8', error_bad_lines=True)


In [27]:
validation_dataframe = pd.read_csv('/content/samsum-validation.csv',engine='python',encoding='utf-8', error_bad_lines=True)

<ipython-input-27-aa3a2d478b16>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  validation_dataframe = pd.read_csv('/content/samsum-validation.csv',engine='python',encoding='utf-8', error_bad_lines=True)


In [28]:
train_data = [(dialogue, summary) for dialogue, summary in zip(dataframe['dialogue'], dataframe['summary'])]

In [29]:
validation_data = [(dialogue, summary) for dialogue, summary in zip(validation_dataframe['dialogue'], validation_dataframe['summary'])]

In [30]:
test_data = [(dialogue, summary) for dialogue, summary in zip(test_dataframe['dialogue'], test_dataframe['summary'])]

In [31]:
train_data[0]

("Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'Amanda baked cookies and will bring Jerry some tomorrow.')

In [32]:
validation_data[0]

("A: Hi Tom, are you busy tomorrow’s afternoon?\r\nB: I’m pretty sure I am. What’s up?\r\nA: Can you go with me to the animal shelter?.\r\nB: What do you want to do?\r\nA: I want to get a puppy for my son.\r\nB: That will make him so happy.\r\nA: Yeah, we’ve discussed it many times. I think he’s ready now.\r\nB: That’s good. Raising a dog is a tough issue. Like having a baby ;-) \r\nA: I'll get him one of those little dogs.\r\nB: One that won't grow up too big;-)\r\nA: And eat too much;-))\r\nB: Do you know which one he would like?\r\nA: Oh, yes, I took him there last Monday. He showed me one that he really liked.\r\nB: I bet you had to drag him away.\r\nA: He wanted to take it home right away ;-).\r\nB: I wonder what he'll name it.\r\nA: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))",
 'A will go to the animal shelter tomorrow to get a puppy for her son. They already visited the shelter last Monday and the son chose the puppy. ')

In [33]:
test_data[0]

("Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye",
 "Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.")

In [34]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [35]:
##
# a function to apply pre-processing steps at a batch level
import torch.nn as nn

def collate_batch(batch):
    dialogue_list, summary_list, length_dialogue, length_summary = [], [], [], []

    # iterate over all reviews in a batch
    for _dialogue, _summary in batch:
        # dialogue preprocessing
        dialogue_processing = torch.tensor(dialogue_pipeline(_dialogue), dtype=torch.int64)
        dialogue_list.append(dialogue_processing)
        length_dialogue.append(dialogue_processing.size(0))
        # summary preprocessing
        summary_processing = torch.tensor(summary_pipeline(_summary), dtype=torch.int64)
        summary_list.append(summary_processing)

        # store the length of processed text
        # this will come handy in future when we want to know the original size of a text (without padding)
        length_summary.append(summary_processing.size(0))

    length_dialogue = torch.tensor(length_dialogue)
    length_summary = torch.tensor(length_summary)
    # pad the processed reviews to make their lengths consistant
    padded_dialogue_list = nn.utils.rnn.pad_sequence(
        dialogue_list, batch_first=True)
    padded_summary_list = nn.utils.rnn.pad_sequence(
        summary_list, batch_first=True)

    # return
    # 1. a list of processed and padded review texts
    # 2. a list of processed labels
    # 3. a list of review text original lengths (before padding)
    return padded_dialogue_list.to(device), padded_summary_list.to(device), length_dialogue.to(device), length_summary.to(device)

In [36]:
dataloader = DataLoader(
    train_data, batch_size=32, shuffle=False, collate_fn=collate_batch
)
dialogue_batch,summary_batch,length_dialogue_batch, length_summary_batch = next(iter(dataloader))

print("summary_batch.shape: ", summary_batch.shape)
print("dialogue_batch.shape: ", dialogue_batch.shape)
print("length_dialogue_batch: ", length_dialogue_batch)
print("length_summary_batch: ", length_summary_batch)

summary_batch.shape:  torch.Size([32, 61])
dialogue_batch.shape:  torch.Size([32, 371])
length_dialogue_batch:  tensor([ 19,  20, 105,  31, 192,  83, 371, 168,  51, 134,  62,  19,  23, 116,
        108, 141,  58,  78,  46,  42, 260, 229,  17, 269,  49,  84,  55, 119,
        114, 111, 334,  88], device='cuda:0')
length_summary_batch:  tensor([11, 12, 16, 20, 29, 29, 45, 34, 16, 41, 17, 16,  7, 20, 20, 20, 37, 15,
        19, 18, 16, 60, 13, 35, 19, 22, 20, 34, 38, 16, 61, 25],
       device='cuda:0')


In [37]:
batch_size = 32

train_dl = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)
valid_dl = DataLoader(
    validation_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)
test_dl = DataLoader(
    test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch
)

### Encoder

In [38]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

In [53]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):
        # src = [batch size, src len]
        # src_len = [batch size]

        embedded = self.dropout(self.embedding(src))
        # Ensure src_len is on CPU and of type int64
        src_len = src_len.to('cpu').long()
        # Pack sequence
        packed_embedded = rnn_utils.pack_padded_sequence(embedded, src_len, batch_first=True, enforce_sorted=False)

        packed_outputs, (hidden, cell) = self.rnn(packed_embedded)

        # Unpack sequence
        outputs, _ = rnn_utils.pad_packed_sequence(packed_outputs, batch_first=True)

        # outputs = [batch size, src len, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        return hidden, cell

### DECODER

In [54]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]

        # Convert input to [batch size, 1] (as we process one word at a time)
        input = input.unsqueeze(1)

        embedded = self.dropout(self.embedding(input))

        # embedded = [batch size, 1, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [batch size, seq len, hid dim]

        prediction = self.fc_out(output.squeeze(1))

        # prediction = [batch size, output dim]

        return prediction, hidden, cell

### Seq2Seq

In [55]:
import random
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_len, trg, teacher_forcing_ratio=0.5):
        # src = [batch size, src len]
        # src_len = [batch size]
        # trg = [batch size, trg len]
        # teacher_forcing_ratio is probability to use teacher forcing

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        # encode the source sequence
        hidden, cell = self.encoder(src, src_len)

        # first input to the decoder is the <sos> tokens
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from our predictions
            top1 = output.argmax(1)

            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[:, t] if teacher_force else top1

        return outputs

In [56]:
import torch.optim as optim
import torch.nn as nn

# Assuming your model, vocabulary, and other parameters are already defined

# Initialize the loss function with ignore_index set to the padding token index
PAD_IDX = vb_summary["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Initialize the optimizer (e.g., Adam)
optimizer = optim.Adam(model.parameters())


In [57]:
INPUT_DIM = len(vb)
OUTPUT_DIM = len(vb_summary)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [58]:
# Create the Encoder and Decoder
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

# Instantiate the Seq2Seq model
model = Seq2Seq(enc, dec, device).to(device)

In [59]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29633, 256)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(16061, 256)
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=16061, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [60]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src, trg, src_len, trg_len = batch
        optimizer.zero_grad()

        output = model(src, src_len, trg)

        # trg = [batch size, trg len]
        # output = [batch size, trg len, output dim]

        output_dim = output.shape[-1]

        # Reshape the outputs and targets so they are [batch size * trg len, output dim]
        output = output.reshape(-1, output_dim)
        trg = trg.reshape(-1)

        loss = criterion(output, trg)

        loss.backward()

        # Clip the gradient to prevent it from exploding
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [61]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, trg, src_len, trg_len = batch

            output = model(src, src_len, trg, 0)  # turn off teacher forcing

            # trg = [batch size, trg len]
            # output = [batch size, trg len, output dim]

            output_dim = output.shape[-1]

            # Reshape the outputs and targets so they are [batch size * trg len, output dim]
            output = output.reshape(-1, output_dim)
            trg = trg.reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [63]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch

In [64]:
import math
import time
from tqdm import tqdm
n_epochs = 10
clip = 1

In [66]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


for epoch in tqdm(range(n_epochs), desc = 'processing'):

    start_time = time.time()

    train_loss = train(model, train_dl, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_dl, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

processing:  10%|█         | 1/10 [02:23<21:35, 143.98s/it]

Epoch: 01 | Time: 2m 23s
	Train Loss: 9.698 | Train PPL: 16289.824
	 Val. Loss: 9.696 |  Val. PPL: 16256.831


processing:  20%|██        | 2/10 [04:47<19:10, 143.80s/it]

Epoch: 02 | Time: 2m 23s
	Train Loss: 9.698 | Train PPL: 16290.121
	 Val. Loss: 9.696 |  Val. PPL: 16256.831


processing:  30%|███       | 3/10 [07:11<16:47, 143.99s/it]

...

Epoch: 08 | Time: 2m 25s
	Train Loss: 9.698 | Train PPL: 16289.985
	 Val. Loss: 9.696 |  Val. PPL: 16256.831


processing:  90%|█████████ | 9/10 [21:37<02:24, 144.44s/it]

Epoch: 09 | Time: 2m 24s
	Train Loss: 9.698 | Train PPL: 16289.145
	 Val. Loss: 9.696 |  Val. PPL: 16256.831


processing: 100%|██████████| 10/10 [24:01<00:00, 144.13s/it]

Epoch: 10 | Time: 2m 23s
	Train Loss: 9.698 | Train PPL: 16289.673
	 Val. Loss: 9.696 |  Val. PPL: 16256.831


In [67]:
torch.save(model.state_dict(), 'model.pth')

In [95]:
test_loss = evaluate(model, test_dl, criterion)

In [96]:
test_loss

9.69612741470337